**CUPED** - метод понижения размерности, основанный на использовании исторических данных и предположения, что текущие данные будут кореллировать с историческими.

Для расчетов надо посчитать новую метрику:

$$
\hat{Y} = Y - \theta (X - \mathbb{E}X)
$$

**Y** - значение метрики во время эксперимента

**X** - ковариата (значение метрики до эксперимента - историческое)

**θ** – коэффициент

$$
\theta = \frac{\text{cov}(Y, X)}{\mathbb{V}X}
$$


**EX** - мат. ожидание (среднее значение метрики)

При подсчете t-теста EX у обоих выборок одинаков, поэтому от него избавляются и  берут:
    
$$ \hat{Y} = Y - \theta X $$

В результате дисперсия новой выборки будет примерно в коэф. корреляция в квадрате раз меньше, чем у исходной.

При этом если тест проводится в течение 4 недель, то ковариату надо брать тоже на 4 неделях до начала теста.

Также нужно рассчитывать **min_sample_size** по формуле MDE, используя cuped-метрику и ее дисперсию.

В этом случае для расчета этой формулы надо взять 4 недели до начала эксп-та для расчета **Y** и 4 недели до этого (8...4 нед до начала эксп-та) для расчета ковариаты.

In [13]:
import numpy as np
import pandas as pd
from scipy import stats
from random import uniform

In [14]:
# генерируем данные (контроль и тест и коррелирующие историч. данные к ним):
control = np.random.normal(20, 8, 20000)
test = np.random.normal(50, 7, 20000)

historical_for_test = np.random.uniform(0.7, 0.8, 20000)*test # создаем историч. данные (ковариату) коррелирующую с экспериментом
historical_for_control = np.random.uniform(0.7, 0.8, 20000)*control # создаем историч. данные (ковариату) коррелирующую с экспериментом

control = pd.DataFrame({'historical_metric':historical_for_control, 'experiment_metric':control})
control['group'] = 'control'
test = pd.DataFrame({'historical_metric':historical_for_test, 'experiment_metric':test})
test['group'] = 'test'

data = pd.concat([test, control])
data.sample(4)

,historical_metric,experiment_metric,group
11552,36.527137,50.315558,test
9627,36.871538,47.374835,test
757,13.678043,19.262900,control
18871,43.173567,60.254838,test


In [15]:
# считаем θ:
x = data['historical_metric']
y = data['experiment_metric']
covariance = np.cov(x, y)[0, 1]
variance = x.var()
theta = covariance / variance

theta

1.322659695493238

In [21]:
# получаем cuped-метрику (ради интереса):
data['cuped_metric'] = data['experiment_metric'] - theta*(data['historical_metric']-data['historical_metric'].mean())

# проверяем равенство средних cuped-метрики и исходной метрики:
print(f"Среднее исходной метрики: {data['experiment_metric'].mean():.3f}")
print(f"Среднее СUPED-метрики: {data['cuped_metric'].mean():.3f}")

# проверяем насколько уменьшилась дисперсия:
print(f"\nДисперсия исходной метрики: {data['experiment_metric'].var():.3f}")
print(f"Дисперсия СUPED-метрики: {data['cuped_metric'].var():.3f}")

Среднее исходной метрики: 35.003
Среднее СUPED-метрики: 35.003

Дисперсия исходной метрики: 282.209
Дисперсия СUPED-метрики: 2.219


In [22]:
# считаем ttest (без вкалада средн. ковариаты но можно брать и с ней):
metric_cuped_control = data.query('group=="control"')['experiment_metric'] - theta * data.query('group=="control"')['historical_metric']
metric_cuped_test = data.query('group=="test"')['experiment_metric'] - theta * data.query('group=="test"')['historical_metric']
_, pvalue = stats.ttest_ind(metric_cuped_control, metric_cuped_test)

pvalue

2.3170459787908874e-61